In [1]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinless_fermion_basis_1d,boson_basis_1d, spin_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [52]:
L = 3
M = 0.01
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [53]:
basis_spin =spin_basis_1d(L=L,S = str(l))
basis_fermions = spinless_fermion_basis_1d(L=L+1,Nf = [0,1,2,3,4])
basis=tensor_basis(basis_spin,basis_fermions)
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
	  0.         |2 2 2>           26         |1 1 1 1>           15  
	  1.         |2 2 2>           26         |1 1 1 0>           14  
	  2.         |2 2 2>           26         |1 1 0 1>           13  
	  3.         |2 2 2>           26         |1 1 0 0>           12  
	  4.         |2 2 2>           26         |1 0 1 1>           11  
	  5.         |2 2 2>           26         |1 0 1 0>           10  
	  6.         |2 2 2>           26         |1 0 0 1>            9  
	  7.         |2 2 2>           26         |1 0 0 0>            8  
	  8.         |2 2 2>           26         |0 1 1 1>            7  
	  9.         |2 2 2>           26         |0 1 1 0>            6  
	 10.         |2 2 2>           26         |0 1 0 1>            5  
	 11.         |2 2 2>           26         |0 1 0 0>            4  
	 12.         |2 2 2>           26         |0 0 1 1>            3  
	 13.         |2 2 2>           26         |0 0

In [54]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [55]:
gauss_law_z_0 = []
gauss_law_z_0.append([1.,0])

gauss_law_f_0 = [[-1.,0]]

gauss_law_0_map = [
    
    ["z|",gauss_law_z_0],
    ["|n",gauss_law_f_0],


    
]

Gauss_law_0  = hamiltonian(gauss_law_0_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_0_q = Gauss_law_0**2

In [56]:
#Gauss_law_1
gauss_law_zm_1 = []
gauss_law_zm_1.append([-1.,0])

gauss_law_z_1 = []
gauss_law_z_1.append([1.,1])

gauss_law_id_1 = [[1.,1]]
gauss_law_f_1 = [[-1.,1]]


gauss_law_1_map = [
    
    ["z|",gauss_law_z_1],
    ["z|",gauss_law_zm_1],
    ["I|",gauss_law_id_1],
    ["|n",gauss_law_f_1],

]

Gauss_law_1  = hamiltonian(gauss_law_1_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_1_q = Gauss_law_1**2

In [57]:
#Gauss_law_2
gauss_law_zm_2 = []
gauss_law_zm_2.append([-1.,1])

gauss_law_z_2 = []
gauss_law_z_2.append([1.,2])


gauss_law_f_2 = [[-1.,2]]
gauss_law_2_map = [
    
    ["z|",gauss_law_z_2],
    ["z|",gauss_law_zm_2],
    ["|n",gauss_law_f_2],


]

Gauss_law_2  = hamiltonian(gauss_law_2_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_2_q = Gauss_law_2**2

In [58]:
#Gauss_law_3

gauss_law_z_3 = []
gauss_law_z_3.append([-1.,2])

gauss_law_id_3 = [[1.,2]]
gauss_law_f_3 = [[-1.,3]]



gauss_law_3_map = [
    
    ["z|",gauss_law_z_3],
    ["I|",gauss_law_id_3],
    ["|n",gauss_law_f_3],



    
]

Gauss_law_3  = hamiltonian(gauss_law_3_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_3_q = Gauss_law_3**2

In [59]:
G = Gauss_0_q+Gauss_1_q+Gauss_2_q+Gauss_3_q

In [60]:
for i in range(basis.Ns):
    if (G.expt_value(basis_vectors[basis.Ns-i-1])<=0.+1e-8) & (G.expt_value(basis_vectors[basis.Ns-i-1])>=0.-1e-8):
        print(i)

3
53
70
201
218
268


In [61]:
kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[M*(-1)**i,i] for i in range(L+1)]
interaction_term = [[-1j/(2*np.sqrt(2)),i,i,i+1] for i in range(L)]

hamiltonian_map = [
    
    ["zz|",kin_energy],
    ["|n",mass_term],
    ["|I",mass_term],

    ["+|+-",interaction_term],
    ["-|-+",interaction_term]
]



Ham_LGT = hamiltonian(hamiltonian_map,dynamic_list = [],basis=basis,**no_checks)
Hamiltonian_LGT = Ham_LGT + 1000*G

In [62]:
ham_matrix = Hamiltonian_LGT.toarray()

In [63]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [64]:
print(check_hermitian(ham_matrix))

True


In [65]:
eigenval,eigenvec = np.linalg.eigh(ham_matrix)

In [66]:
print(eigenval[eigenval<=1e2])

[-0.74840735  0.19473274  0.5         0.77641838  1.30716476  1.97009147]


In [67]:
eig, eigv = Ham_LGT.eigh()

In [51]:
print(eig)

[-2.25847284e+00 -1.80916457e+00 -1.80916457e+00 -1.80323692e+00
 -1.67414858e+00 -1.66744527e+00 -1.66744527e+00 -1.36068991e+00
 -1.35006857e+00 -1.35006857e+00 -1.23056193e+00 -1.23056193e+00
 -1.21755071e+00 -1.21755071e+00 -1.21755071e+00 -1.21755071e+00
 -1.18485703e+00 -1.18485703e+00 -1.09658795e+00 -1.09658795e+00
 -1.07701159e+00 -1.07701159e+00 -1.07701159e+00 -8.97543982e-01
 -8.18045592e-01 -8.18045592e-01 -7.88761348e-01 -7.63523327e-01
 -7.63523327e-01 -7.34087277e-01 -7.34087277e-01 -7.30577082e-01
 -7.30577082e-01 -6.77445270e-01 -6.77445270e-01 -6.33084542e-01
 -6.33084542e-01 -6.33084542e-01 -5.96865229e-01 -5.96865229e-01
 -5.95937375e-01 -5.95937375e-01 -5.95937375e-01 -5.95937375e-01
 -5.87011593e-01 -5.87011593e-01 -5.87011593e-01 -5.87011593e-01
 -5.32244367e-01 -5.32244367e-01 -5.27924686e-01 -5.27924686e-01
 -4.90000000e-01 -4.90000000e-01 -4.80000000e-01 -3.46407086e-01
 -3.46407086e-01 -2.73523327e-01 -2.73523327e-01 -2.67069896e-01
 -2.67069896e-01 -2.27550